#### Import packages

In [8]:
# import torch.multiprocessing as mp
# mp.set_start_method('spawn') 
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import math
import shutil
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import log_loss
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import random
import cv2
import copy


seed = 0
# random.seed(seed)
np.random.seed(seed)
# from utils_3 import *
# from utils import *
# from utils_diff_ch import *
from utils_diff_ch_scaled import *

#### Load and Prepare Data

In [2]:
# def scale_range (input_, max_, min_):
#     input_[:,0,:,:] += -(-45.594448)
#     input_[:,0,:,:] /= 82.68497099999999 / (max_ - min_)
#     input_[:,0,:,:] += min_
#     input_[:,1,:,:] += -(-49.083500000000001)
#     input_[:,1,:,:] /= 69.686413000000002 / (max_ - min_)
#     input_[:,1,:,:] += min_
#     return input_

def scale_range (input_, max_, min_):
    return input_

In [3]:
data = pd.read_json('./Data/train.json', dtype={'inc_angle': str})
test = pd.read_json('./Data/test.json', dtype={'inc_angle': str})

In [4]:
data['band_1'] = data['band_1'].apply(lambda x : np.array(x).reshape(75, 75))
data['band_2'] = data['band_2'].apply(lambda x : np.array(x).reshape(75, 75))

test['band_1'] = test['band_1'].apply(lambda x : np.array(x).reshape(75, 75))
test['band_2'] = test['band_2'].apply(lambda x : np.array(x).reshape(75, 75))

test['not_machine_generated'] = test['inc_angle'].apply(lambda x: len(str(x).split('.')[1])) <= 4
data['inc_angle'] = pd.to_numeric(data['inc_angle'], errors='coerce')
test['inc_angle'] = pd.to_numeric(test['inc_angle'], errors='coerce')

In [5]:
# Impute missing incidence angles
data_impute = data.dropna(axis=0, how='any')
data_impute_b1 = np.array([im.reshape(75*75) for im in data_impute.loc[:,'band_1']])
data_impute_b2 = np.array([im.reshape(75*75) for im in data_impute.loc[:,'band_2']])
data_impute_X = np.hstack(np.array((data_impute_b1, data_impute_b2)))
data_impute_y = np.array(data_impute[['inc_angle']])

data_impute_prec = data[~data.index.isin(data_impute.index.values)]
data_impute_b1_prec = np.array([im.reshape(75*75) for im in data_impute_prec.loc[:,'band_1']])
data_impute_b2_prec = np.array([im.reshape(75*75) for im in data_impute_prec.loc[:,'band_2']])
data_impute_X_prec = np.hstack(np.array((data_impute_b1_prec, data_impute_b2_prec)))

modelImpute = LinearRegression()
modelImpute = modelImpute.fit(data_impute_X, data_impute_y)
data_impute_prec['inc_angle'] = modelImpute.predict(data_impute_X_prec)

data.loc[data_impute_prec.index] = data_impute_prec

/home/arsh/anaconda3/envs/torch/lib/python3.6/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


#### KFold

In [6]:
band_1_KF = np.concatenate([im for im in data['band_1']]).reshape(-1, 75*75)
band_2_KF = np.concatenate([im for im in data['band_2']]).reshape(-1, 75*75)
# band_3_KF = scale_range(band_1_KF/band_2_KF, -1, 1)
X_KF = scale_range(np.concatenate((band_1_KF, band_2_KF, \
                                   np.array(data['inc_angle']).reshape((len(data), 1))), axis=1), -1, 1)

y_KF = data['is_iceberg'].values.astype(np.float32)

band_1_test = np.concatenate([im for im in test['band_1']]).reshape(-1, 75*75)
band_2_test = np.concatenate([im for im in test['band_2']]).reshape(-1, 75*75)
# band_3_test = scale_range(band_1_test/band_2_test, -1, 1)
X_test = scale_range(np.concatenate((band_1_test, band_2_test,
                                    np.array(test['inc_angle']).reshape((len(test), 1))), axis=1), -1, 1)

In [15]:
X, X_val, y, y_val = train_test_split(X_KF, y_KF, test_size=0.2)

In [18]:
parameters = {'kernel':('linear', 'rbf', 'poly', 'sigmoid'), 'C':[1, 10, 100],
             'degree' : [2, 3], 'probability' : [True]}
svm = SVC()
clf = GridSearchCV(svm, parameters, n_jobs=-1)
clf = clf.fit(X, y)

In [37]:
svm = SVC(kernel='linear', C=1000, probability=True)
new_clf = svm.fit(X, y)

In [39]:
log_loss(y, new_clf.predict_proba(X)[:,1])

0.13288798165080765

In [7]:
kfold = 5
kfold_scores = []

results = []
fold_predictions = []
sss = KFold(n_splits=kfold, random_state=0)
for i, (train_index, val_index) in enumerate(sss.split(X_KF, y_KF)):
    print('Fold [%d/%d]' % (i+1, kfold))
    X_train_KF = X_KF[train_index]
    X_valid_KF = X_KF[val_index]
    y_train_KF, y_valid_KF = y_KF[train_index], y_KF[val_index]
    
    # SVM
    clf = SVC(C=10, probability=True)
    clf = clf.fit(X_train_KF, y_train_KF)
    
    val_pred = clf.predict_proba(X_valid_KF)[:,0]
    score = log_loss(y_valid_KF, val_pred)
    print('Val Score : %f' % (score))
    fold_predictions.append(val_pred)
    kfold_scores.append(score)
    
    # Test Predict
    results.append(np.array(clf.predict_proba(X_test)[:,0]))

Fold [1/5]
Val Score : 0.693607
Fold [2/5]
Val Score : 0.693232
Fold [3/5]


KeyboardInterrupt: 

In [79]:
np.array(kfold_scores).mean()

0.21121035516262054

In [42]:
np.array(kfold_scores).std()

0.038812024497466201

In [33]:
sub_num = 54
os.makedirs("./Models/Sub "+str(sub_num))
np.savetxt("./Models/Sub "+str(sub_num)+"/results.csv", np.array(results), delimiter=",")
np.savetxt("./Models/Sub "+str(sub_num)+"/cv_fold_results.csv", np.concatenate(fold_predictions), delimiter=",")
sub = pd.read_csv('./Data/sample_submission.csv')
sub['is_iceberg'] = np.array(results).mean(axis=0)
sub.to_csv('./Submissions/Sub '+str(sub_num)+' - 5-fold _ Val-1670.csv', index=False)

In [3]:
sub = pd.read_csv('./Data/sample_submission.csv')

In [4]:
sub1 = pd.read_csv('./Submissions/Sub 46 - 5-fold _ Val-1563.csv')['is_iceberg']
sub2 = pd.read_csv('./Submissions/Sub 47 - 5-fold _ Val-1679.csv')['is_iceberg']
sub3 = pd.read_csv('./Submissions/Sub 48 - 5-fold _ Val-1583.csv')['is_iceberg']
sub4 = pd.read_csv('./Submissions/Sub 22 - 5-fold _ Val-1571.csv')['is_iceberg']
# sub5 = pd.read_csv('./Submissions/Sub 8 - 5-fold _ Val-1479.csv')['is_iceberg']
# sub6 = pd.read_csv('./Submissions/Sub 16 - 5-fold _ Val-1422.csv')['is_iceberg']

In [5]:
results = (np.array(sub1)*np.array(sub2)*np.array(sub3)*np.array(sub4))**(1/4)
sub['is_iceberg'] = np.array(results)
sub.to_csv('./Submissions/Sub 49 - Ensemble_22_46_47_48 - Geometric Mean.csv', index=False)

#### Error Analysis

In [24]:
def accuracy(loader):
    iceNet.eval()
    correct = 0
    total = 0
    loss = []
    inc_features = []
    inc_feature_angles = []
    inc_labels = []
    for features, features_angle, labels in loader:
        features = Variable(features, volatile=True).cuda()
        features_angle = Variable(features_angle, volatile=True).cuda()
        labels = Variable(labels, volatile=True).cuda()
        outputs = iceNet(features, features_angle)
        _loss = criterion(outputs, labels)
        loss.append(_loss)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels.data).sum()
        inc_features.append(features.data.cpu().numpy()[np.where((predicted!=labels.data).cpu().numpy())])
        inc_feature_angles.append(features_angle.data.cpu().numpy()[np.where((predicted!=labels.data).cpu().numpy())])
        inc_labels.append(labels.data.cpu().numpy()[np.where((predicted!=labels.data).cpu().numpy())])
        
    return np.mean(loss).data[0], (100 * correct / total), [inc_features, inc_feature_angles, inc_labels]

In [25]:
kfold = 5
kfold_scores = []

results = []
sss = KFold(n_splits=kfold, random_state=0)
for i, (train_index, test_index) in enumerate(sss.split(X_KF[0], X_KF[1], y_KF)):
    if i == 2:
        X_train_KF, X_valid_KF = [X_KF[0][train_index], X_KF[1][train_index]], [X_KF[0][test_index], X_KF[1][test_index]]
        y_train_KF, y_valid_KF = y_KF[train_index], y_KF[test_index]

        # Define model
        iceNet = net().apply(weight_init).cuda()

        # Loss and Optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(iceNet.parameters(), lr=learning_rate, weight_decay=0.0001)

        # Data Loader
        train_dataset_KF = icebergDataset(X_train_KF, y_train_KF, transform=True)
        val_dataset_KF = icebergDataset(X_valid_KF, y_valid_KF)

        train_loader_KF = torch.utils.data.DataLoader(dataset=train_dataset_KF, batch_size=batch_size, shuffle=True)
        val_loader_KF = torch.utils.data.DataLoader(dataset=val_dataset_KF, batch_size=batch_size, shuffle=False)

        print('Fold [%d/%d]' % (i+1, kfold))
        # Train
        best_prec1 = 1
        for epoch in range(num_epochs):
            epoch_train_loss = []
            for idx, (features, features_angle, labels) in enumerate(train_loader_KF):
                iceNet.train()
                features = Variable(features).cuda()
                features_angle = Variable(features_angle).cuda()
                labels = Variable(labels).cuda()
                # Forward + Backward + Optimize
                optimizer.zero_grad()
                outputs = iceNet(features, features_angle)
                loss = criterion(outputs, labels)
                epoch_train_loss.append(loss)
                loss.backward()
                optimizer.step()
            
            acc = accuracy(val_loader_KF)
            prec1 = acc[0]

            # Save best model
            is_best = prec1 < best_prec1
            best_prec1 = min(prec1, best_prec1)
            if is_best:
                incorrect = acc[2]
print(best_prec1)

/home/arsh/anaconda3/envs/torch/lib/python3.6/site-packages/ipykernel/__main__.py:8: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


Fold [3/5]
0.2119673639535904


In [26]:
incorrect[0] = np.vstack(incorrect[0])
incorrect[1] = np.vstack(incorrect[1])
incorrect[2] = np.hstack(incorrect[2])

In [46]:
def randomZoomOut(im, zoom_ratio=(-0.5, 0.5), u=0.5):
    if np.random.random() < u:
        height, width = im[0].shape
        
        zoom = height*np.random.uniform(zoom_ratio[0], zoom_ratio[1])
        if int(zoom)<0:
            zoom = abs(int(zoom))
            im_0 = cv2.resize(im[0, zoom//2:-zoom//2, zoom//2:-zoom//2], (height, width))
            im_1 = cv2.resize(im[1, zoom//2:-zoom//2, zoom//2:-zoom//2], (height, width))
#            im_2 = cv2.resize(im[2, zoom//2:-zoom//2, zoom//2:-zoom//2], (height, width))
#            im_3 = cv2.resize(im[3, zoom//2:-zoom//2, zoom//2:-zoom//2], (height, width))
            return np.stack((im_0,im_1))
        
        if zoom<0:
            zoom = 0
        im_0 = cv2.resize(cv2.copyMakeBorder(im[0],int(zoom//2),int(zoom//2),int(zoom//2),int(zoom//2),
                                   cv2.BORDER_REFLECT101), (height, width))
        im_1 = cv2.resize(cv2.copyMakeBorder(im[1],int(zoom//2),int(zoom//2),int(zoom//2),int(zoom//2),
                                   cv2.BORDER_WRAP), (height, width))
#        im_2 = cv2.resize(cv2.copyMakeBorder(im[2],int(zoom//2),int(zoom//2),int(zoom//2),int(zoom//2),
#                                   cv2.BORDER_REPLICATE), (height, width))
#        im_3 = cv2.resize(cv2.copyMakeBorder(im[3],int(zoom//2),int(zoom//2),int(zoom//2),int(zoom//2),
#                                   cv2.BORDER_REPLICATE), (height, width))
        return np.stack((im_0,im_1))
    
    return im

In [47]:
for idx, im in enumerate(incorrect[0]):
    f, axarr = plt.subplots(2, 2, figsize=(10,8))
    plt.title('Label = '+str(incorrect[2][idx])+', Inc Angle = '+str(incorrect[1][idx][0]))
    zm = randomZoomOut(im, (0, 0.5), 1)
    axarr[0, 0].imshow(im[0], cmap='gray')
    axarr[0, 1].imshow(im[1], cmap='gray')
    axarr[1, 0].imshow(zm[0], cmap='gray')
    axarr[1, 1].imshow(zm[1], cmap='gray')
    plt.show()

error: /home/travis/miniconda/conda-bld/conda_1486587069159/work/opencv-3.1.0/modules/core/src/copy.cpp:928: error: (-5) Unknown/unsupported border type in function borderInterpolate
